In [280]:
############
# Enviroment set-up
############
import string
import os
import re
from pprint import pprint
from collections import defaultdict
import postgres
import poker
from IPython.core.display import display, HTML

position_mapping = {
    "Dealer": "BU",
    "Small Blind": "SB",
    "Big Blind": 'BB',
    "UTG": "UTG",
    "UTG+1": 'LJ',
    "UTG+2": 'HJ'
}

C:\Users\David\AppData\Local\Temp\ipykernel_15880\3874190313.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [281]:
HTML("""
<style>
   td {
      /* Make cells same width and height and centered */
      width: 25px;
      height: 25px;
      text-align: center;
      vertical-align: middle;
   }
   td.pair {
      background: #aaff9f;
   }
   td.offsuit {
      background: #bbced3;
   }
   td.suited {
      background: #e37f7d;
   }
</style>
""")

# Ignition Hand Parsing

Couple things:
   - Processes iggy hands in bulk per table (per file)
   - No post flop action: Board = None
   - Board is always 5 len otherwise; so no turn action means 2 empty strings after
       - Just how iggy does it...
   - Hand is a tuple that's added to each player's entry 

In [282]:
def find_pos(line):
    pos = None
    for key in position_mapping:
        if key in line:
            pos = position_mapping[key]
    if pos:
        return pos

def get_bb(line, bb):
    return round(float(re.findall("\$\d+(?:\.\d+)?", line)[0][1:])/bb)

In [283]:
############
# Parse Ignition hands
# 
# Ignition hands are broken into large files; each file is a series of hands that belong to that table
############

def get_stakes(hand):
    sb, bb = None, None
    for line in hand:
        if ': Small Blind $' in line:
            sb = float(re.findall("\$\d+(?:\.\d+)?", line)[0][1:])
        if ': Big blind $' in line:
            bb = float(re.findall("\$\d+(?:\.\d+)?", line)[0][1:])
            
        if sb and bb:
            break
    
    return ((sb, bb), bb)
            

def get_players(hand, bb):
    temp_dict = defaultdict(list)
    player_list = []
    
    for line in hand:
        if "in chips" in line:
            bigs = get_bb(line, bb)
            pos = find_pos(line)
            player = 'Hero' if "ME" in line else ''
            
            temp_dict[pos] = [pos, player, bigs]
            
        if "Card dealt" in line:
            cards = tuple(re.findall("\[(.*?)\]", line)[0].split(" "))
            if cards[0] == 'ME':
                cards = tuple(re.findall("\[(.*?)\]", line)[1].split(" "))
            temp_dict[find_pos(line)].append(cards)
            
            
    return list(temp_dict.values())
                
                

def get_board(hand):
    for line in hand:
        if 'Board' in line:
            return re.findall("\[(.*?)\]", line)[0].split(" ")
            
def get_action(hand, bb):
    action = []
    
    street_action = []
    for line in hand:
        if 'Calls' in line:
            street_action.append(find_pos(line) + "c" + str(get_bb(line, bb)))
        if 'Raises' in line or 'raise' in line:
            street_action.append(find_pos(line) + "r" + str(get_bb(line, bb)))
            continue
        if 'Folds' in line:
            street_action.append(find_pos(line) + "f")
        if 'Checks' in line:
            street_action.append(find_pos(line) + "x")
        if 'Bets' in line or 'All-in' in line:
            street_action.append(find_pos(line) + "b" + str(get_bb(line, bb)))
        
        if "FLOP" in line or "TURN" in line or "RIVER" in line:
            if len(street_action) > 0:
                action.append(street_action) 
            street_action = []
        if "SUMMARY" in line:
            if len(street_action) > 0:
                action.append(street_action)
            break
            
    return action

def get_winner(hand):
    winners = []
    wsd = None
    for line in hand[::-1]:
        if 'Hand result' in line or re.findall("\$\d+(?:\.\d+)?", line):
            winner = find_pos(line)
            wsd = False if "Does not show" in line else True
            winners.append(winner)
        
        if "Total Pot" in line or "Hand result" in line:
            if not wsd:
                wsd = False
            break
            
    return (winners, wsd)
    
            

def get_hand_number(hand):
    return re.findall("\d+", hand[0])[0]


def build_hand(file_name):
    #Builds all hands for given file
    file = open(file_name, "r").read()
    hands = re.split(r'\n{2,}', file)
    hands = hands[:len(hands)-1]

    ret = {}
    for hand in hands:
        hand = re.split(r'\n{1,}', hand)
        
        hand_dict = {}
        hand_dict['Stakes'], bb = get_stakes(hand) 
        hand_dict['Players'] = get_players(hand, bb)
        hand_dict['Board'] = get_board(hand)
        hand_dict['Action'] = get_action(hand, bb)
        hand_dict['Winner'], hand_dict["WTSD"] = get_winner(hand)
        ret[get_hand_number(hand)] = hand_dict
    
    return ret


####
# TEST
####
#for root, dirs, files in os.walk(r"C:\Users\David\Ignition Casino Poker\Hand History\580402808201", topdown=False):0
#    for file in files:
#        print(file)
#        pprint(build_hand(root + "\\" + file))

# 5nl P/F Analysis

In [295]:
hands=[]

for root, dirs, files in os.walk(r"C:\Users\David\Ignition Casino Poker\Hand History\580402808201", topdown=False):
    for file in files:
        if "ZonePoker" in file and ".05" in file:
            hands.append(build_hand(root + "\\" + file))
            

In [296]:
sum_hands = 0
for i in hands:
    sum_hands+=len(i.keys())

print("Number of 5nl hands:", sum_hands)

Number of 5nl hands: 2080


In [297]:
pprint(hands)

[{'4474450410': {'Action': [['UTGf',
                             'LJc1',
                             'HJf',
                             'BUf',
                             'SBf',
                             'BBr50',
                             'LJf']],
                 'Board': None,
                 'Players': [['BB', 'Hero', 51, ('Kc', 'As')],
                             ['UTG', '', 99, ('4h', '2d')],
                             ['LJ', '', 50, ('8d', 'Jh')],
                             ['HJ', '', 127, ('Qs', '2c')],
                             ['BU', '', 224, ('Jc', '6c')],
                             ['SB', '', 103, ('3c', '7c')]],
                 'Stakes': (0.02, 0.05),
                 'WTSD': True,
                 'Winner': ['BB', 'BB']},
  '4474450562': {'Action': [['UTGf', 'LJf', 'HJf', 'BUr3', 'SBf', 'BBc2'],
                            ['BBb1', 'BUc1'],
                            ['BBb1', 'BUf']],
                 'Board': None,
                 'Players': [['BB'

                 'Players': [['HJ', '', 864, ('5s', '3h')],
                             ['BU', 'Hero', 49, ('4h', '9d')],
                             ['SB', '', 130, ('Js', 'Jh')],
                             ['BB', '', 44, ('Qh', 'Ts')],
                             ['UTG', '', 140, ('3d', '9h')],
                             ['LJ', '', 189, ('Ad', '8c')]],
                 'Stakes': (0.02, 0.05),
                 'WTSD': True,
                 'Winner': ['BB']},
  '4474478219': {'Action': [['UTGc1',
                             'LJf',
                             'HJf',
                             'BUr3',
                             'SBc3',
                             'BBf',
                             'UTGf'],
                            ['SBb12', 'BUf']],
                 'Board': None,
                 'Players': [['UTG', '', 135, ('Th', 'Jh')],
                             ['LJ', '', 84, ('2d', 'Qh')],
                             ['HJ', '', 100, ('4s', '3c')],
           

                            ['BBb134', 'HJf', 'BUf']],
                 'Board': None,
                 'Players': [['UTG', '', 118, ('5h', 'Ac')],
                             ['LJ', 'Hero', 53, ('6d', '4d')],
                             ['HJ', '', 91, ('As', 'Th')],
                             ['BU', '', 50, ('5s', '4s')],
                             ['SB', '', 73, ('Tc', '8d')],
                             ['BB', '', 151, ('Ks', 'Kc')]],
                 'Stakes': (0.02, 0.05),
                 'WTSD': True,
                 'Winner': ['BB']},
  '4474513017': {'Action': [['UTGf', 'LJf', 'HJf', 'BUf', 'SBf']],
                 'Board': None,
                 'Players': [['SB', '', 113, ('8c', '4h')],
                             ['BB', 'Hero', 53, ('Qc', 'Kc')],
                             ['UTG', '', 151, ('8s', 'Kh')],
                             ['LJ', '', 531, ('Jd', '6d')],
                             ['HJ', '', 186, ('Qh', '4c')],
                             ['BU', '', 

In [298]:
def _get_hero_pos(players):
    for player in players:
        if player[1] == 'Hero':
            return player[0]

def _hero_raise_pre(pre_action, pos):
    for action in pre_action:
        if (pos + "r" in action or pos + "b" in action):
        #if (pos + "f" in action):
            return True
        
    return False


def get_hero_VPIPed(hands):
    vpiped_hands = []
    for hh in hands:
        for hand in hh.values():
            pos = _get_hero_pos(hand['Players'])
            if _hero_raise_pre(hand['Action'][0], pos):
                vpiped_hands.append(hand)
    return vpiped_hands

def get_hero_VPIPed_hands(hands):
    vpiped_hands = []
    for hand in hands:
        for player in hand['Players']:
            if player[1] == 'Hero':
                vpiped_hands.append(player[3])
    return vpiped_hands
            
        
            
hero_vpip = get_hero_VPIPed(hands)
hero_vpip_hands = get_hero_VPIPed_hands(hero_vpip)

In [299]:
pprint(hero_vpip)

[{'Action': [['UTGf', 'LJc1', 'HJf', 'BUf', 'SBf', 'BBr50', 'LJf']],
  'Board': None,
  'Players': [['BB', 'Hero', 51, ('Kc', 'As')],
              ['UTG', '', 99, ('4h', '2d')],
              ['LJ', '', 50, ('8d', 'Jh')],
              ['HJ', '', 127, ('Qs', '2c')],
              ['BU', '', 224, ('Jc', '6c')],
              ['SB', '', 103, ('3c', '7c')]],
  'Stakes': (0.02, 0.05),
  'WTSD': True,
  'Winner': ['BB', 'BB']},
 {'Action': [['UTGr3', 'LJf', 'HJc3', 'BUf', 'SBr15', 'BBf', 'UTGc12', 'HJf'],
             ['SBb10', 'UTGc10'],
             ['SBx', 'UTGx'],
             ['SBb73', 'UTGf']],
  'Board': ['Ts', 'Ah', '4c', 'Kd', '6c'],
  'Players': [['BU', '', 689, ('5d', 'Kh')],
              ['SB', 'Hero', 99, ('9h', '9s')],
              ['BB', '', 109, ('Jd', '6h')],
              ['UTG', '', 101, ('7s', '7d')],
              ['LJ', '', 88, ('7h', '5c')],
              ['HJ', '', 51, ('As', '3s')]],
  'Stakes': (0.02, 0.05),
  'WTSD': True,
  'Winner': ['SB', None]},
 {'Action':

In [300]:
neat_vpip_hands = set(())
for hand in hero_vpip_hands:
    if(hand[0][0] == hand[1][0]):
        neat_vpip_hands.add(hand[0][0]*2)
    else:
        if(hand[0][1] == hand[1][1]):
            neat_vpip_hands.add(hand[0][0]+hand[1][0]+"s")
        else:
            neat_vpip_hands.add(hand[0][0]+hand[1][0]+"o")
hero_vpip_range = poker.Range(" ".join(neat_vpip_hands))

In [301]:
display(HTML(hero_vpip_range.to_html()))

AA,AKs,AQs,AJs,ATs,A9s,A8s,A7s,A6s,A5s,A4s,A3s,A2s
AKo,KK,KQs,KJs,KTs,,,,,,,,
AQo,KQo,QQ,QJs,,,,,,,,,
AJo,KJo,,JJ,,,,,,,,,
ATo,,,,TT,,,,,,,,
,,,,,99,,,,,,,
,,,,,,88,,,,,,
,,,,,,,77,,,,,
,,,,,,,,66,,,,
,,,,,,,,,55,,,
,,,,,,,,,,44,,


In [307]:
def _get_caller_hand(players, pos):
    for player in players:
        if(player[0] == pos):
            return player[3]


def get_villian_vpip_hands_against_shove(hands):
    villian_vpip_range = set(())
    villian_vpip_hands = []
    
    for hand in hands:
        post_shove = False
        hero_pos = _get_hero_pos(hand["Players"])
        for action in hand['Action'][0]:
            if hero_pos + 'r' in action:
                post_shove = True
            if post_shove and 'c' in action:
                pos = action.split('c')[0]
                villian_vpip_hands.append(hand)
                villian_vpip_range.add(_get_caller_hand(hand['Players'], pos))

    return (villian_vpip_range, villian_vpip_hands)

villian_vpip, villian_vpip_hands = get_villian_vpip_hands_against_shove(hero_vpip)

In [303]:
def _create_neat_range(set_of_hands):
    neat_set = set(())
    for hand in set_of_hands:
        if(ord(hand[0][0]) < ord(hand[1][0])):
            if(hand[0][1] == hand[1][1]):
                neat_set.add(hand[1][0] + hand[0][0] + "s")
            else:
                neat_set.add(hand[1][0] + hand[0][0] + "o")
        elif(ord(hand[0][0]) > ord(hand[1][0])):
            if(hand[0][1] == hand[1][1]):
                neat_set.add(hand[0][0] + hand[1][0] + "s")
            else:
                neat_set.add(hand[0][0] +hand[1][0] + "o")
        else:
            neat_set.add(hand[0][0]*2)
    return neat_set
        
                
    
neat_vpip_hands = _create_neat_range(villian_vpip)
villan_neat_vpip_hands = poker.Range(" ".join(neat_vpip_hands))

display(HTML(villan_neat_vpip_hands.to_html()))

AA,AKs,,,,A9s,,,A6s,A5s,A4s,,A2s
AKo,KK,KQs,,KTs,,,,,,,,
AQo,,QQ,QJs,,,,Q7s,,,,,
AJo,KJo,,JJ,,,,,,,,,
,,,,,T9s,,,,,,,
,,,,,99,,,,,,,
,,,,,,88,,,,,,
,,,,,,87o,77,,,,,
,,,,,,,,66,,,,
,,,,,,,,,,,,
A4o,,,,,,,,,,44,,


In [304]:
print(villan_neat_vpip_hands)

JJ+, 99-66, 44, AKs, A9s, A6s-A4s, A2s, KQs, KTs, QJs, Q7s, T9s, AJo+, A4o, A2o, KJo, 87o


In [305]:
print(len(hero_vpip))

232


In [316]:
def _get_caller_stack_size(players, pos):
    for player in players:
        if(player[0] == pos):
            return player[2]

def find_v_stack_size(hands):
    v_stack_hand = []
    for hand in hands:
        post_shove = False
        hero_pos = _get_hero_pos(hand["Players"])
        for action in hand['Action'][0]:
            if hero_pos + 'r' in action:
                post_shove = True
            if post_shove and 'c' in action:
                pos = action.split('c')[0]
                v_stack_hand.append((_get_caller_hand(hand['Players'], pos),  _get_caller_stack_size(hand['Players'], pos)))
    return v_stack_hand

v_stack_hand = find_v_stack_size(villian_vpip_hands)

v_hand_low_stack = []
v_hand_high_stack = []
for i in v_stack_hand:
    if(i[1] > 75):
        v_hand_high_stack.append(i[0])
    else:
        v_hand_low_stack.append(i[0])

v_hand_low_stack = _create_neat_range(v_hand_low_stack)
v_hand_high_stack = _create_neat_range(v_hand_high_stack)

In [320]:
display(HTML(poker.Range(" ".join(v_hand_high_stack)).to_html()))

AA,AKs,,,,A9s,,,A6s,,,,A2s
AKo,KK,KQs,,KTs,,,,,,,,
AQo,,QQ,,,,,Q7s,,,,,
,KJo,,JJ,,,,,,,,,
,,,,,T9s,,,,,,,
,,,,,,,,,,,,
,,,,,,88,,,,,,
,,,,,,87o,77,,,,,
,,,,,,,,66,,,,
,,,,,,,,,,,,
A4o,,,,,,,,,,44,,
